# Can we beat Welch's t-test?

The current version of this script is super-stupid. But to do any interesting research, you have to plant
a few small acorns, and this is the smallest of acorns.

Suppose you have to evaluate NUMBER_OF_EXPERIMENTS experiments very small experiment. By a weird
coincidence, they all have the same number of participants. There are CONTROL_GROUP_SIZE patients
getting a placebo, and EXPERIMENT_GROUP_SIZE getting a drug.

Unfortunately approximately PROPORTION_OF_EXPERIMENTS_THAT_DO_NOTHING of those drugs actually have no effect either.

You want to create a classifier to determine whether or not the experimental drug worked.

Also, amazingly, the control group and the experimental group metrics are normally distributed. Obviously, the
experiments where the drug is useless have the same mean and standard deviation. In the experiments where
the drug is useful, they differ. The means of these datasets are uniformly distributed between 0-10. The standard deviations are uniformly distributed by 1-4.

Being a well-trained statistician with years of experience, you use Welch's t-test.

Your offsider is a clueless data scientist whose sole training consisted of reading the sklearn documentation.

You have a future self who will implement a beautifully trained transformer or fully-connected neural network. (I haven't done that in this code yet. One day soon.)

Who wins?

In [1]:
CONTROL_GROUP_SIZE=5
EXPERIMENT_GROUP_SIZE=6
PROPORTION_OF_EXPERIMENTS_THAT_DO_NOTHING=0.66
NUMBER_OF_EXPERIMENTS=100000

In [2]:
import scipy
import scipy.stats
import collections
import pandas
import sklearn.model_selection
import sklearn.svm
import sklearn.dummy
import sklearn.metrics
import sklearn.tree
import sklearn.ensemble
import sklearn.neighbors

In [3]:
Experiment = collections.namedtuple("Experiment",
                                    ["ControlLoc", "ControlScale", "ControlValues",
                                   "ExperimentLoc", "ExperimentScale", "ExperimentValues", 
                                   "ShouldShowResult"])

In [4]:
def generate_experiment():
    null_experiment = scipy.stats.uniform.rvs() <= PROPORTION_OF_EXPERIMENTS_THAT_DO_NOTHING
    control_loc = scipy.stats.uniform.rvs(loc=0, scale=10)
    control_scale = scipy.stats.uniform.rvs(loc=1, scale=4)
    if null_experiment:
        experiment_loc = control_loc
        experiment_scale = control_scale
    else:
        experiment_loc = scipy.stats.uniform.rvs(loc=0, scale=10)
        experiment_scale = scipy.stats.uniform.rvs(loc=1, scale=4)
    control_values = scipy.stats.norm.rvs(loc=control_loc, scale=control_scale, size=CONTROL_GROUP_SIZE)
    experiment_values = scipy.stats.norm.rvs(loc=experiment_loc, scale=experiment_scale, size=EXPERIMENT_GROUP_SIZE)
    return Experiment(ControlLoc=control_loc, 
                      ControlScale=control_scale,
                     ControlValues=control_values,
                     ExperimentLoc=experiment_loc,
                     ExperimentScale=experiment_scale,
                     ExperimentValues=experiment_values,
                     ShouldShowResult=not(null_experiment))

def generate_experiments(number_of_experiments):
    return [generate_experiment() for n in range(number_of_experiments)]

In [5]:
def create_feature_dataframe(experiments):
    records = []
    for experiment in experiments:
        record = {}
        for i in range(CONTROL_GROUP_SIZE):
            record[f"c{i}"] = experiment.ControlValues[i]
        for i in range(EXPERIMENT_GROUP_SIZE):
            record[f"x{i}"] = experiment.ExperimentValues[i]
        records.append(record)
    return pandas.DataFrame.from_records(records)

def create_target_series(experiments):
    return pandas.Series([x.ShouldShowResult for x in experiments])

In [6]:
experiments = generate_experiments(NUMBER_OF_EXPERIMENTS)
create_feature_dataframe(experiments)

,c0,c1,c2,c3,c4,x0,x1,x2,x3,x4,x5
0,1.948896,1.842204,3.356506,8.396645,2.716596,4.087990,3.329450,1.303297,4.856509,-4.646989,5.671129
1,5.700836,6.696139,6.077726,6.980054,7.844831,5.705246,4.207307,5.960568,7.485636,-1.021265,2.424254
2,1.425377,3.246844,9.847236,2.468435,1.735519,1.397936,-0.017366,-0.525483,-0.466242,-0.234993,-1.120525
3,10.062429,8.309879,11.900366,-0.176416,12.084270,8.167442,6.817105,9.620867,6.982023,10.886390,0.855377
4,8.330335,13.145230,7.705072,6.980150,11.142246,12.263262,6.225753,9.118882,13.347363,11.145861,8.717043
...,...,...,...,...,...,...,...,...,...,...,...
99995,7.566206,7.794550,5.125132,7.112215,6.144150,4.824090,5.676929,5.495430,3.152568,8.311746,5.230516
99996,0.712997,3.747413,0.277560,2.314402,1.556554,15.139612,9.855118,12.038876,8.462546,3.941723,11.352306
99997,2.857010,6.830576,6.941413,3.484801,3.516044,5.078779,2.749225,2.474191,2.507828,5.688720,1.521244
99998,-0.263602,-0.387263,-0.438490,4.021054,1.542363,6.770586,2.391286,8.858161,2.462570,9.267146,-1.742045


In [7]:
create_target_series(experiments).mean()

0.34097

In [8]:
class WelchTTest:
    def __init__(self):
        pass
    def fit(self, X,y, weights=[]):
        pass
    def predict(self, Xs):
        answer = []
        for experiment in Xs:
            outcome = scipy.stats.ttest_ind(experiment.ControlValues, experiment.ExperimentValues, equal_var=False)
            if outcome.pvalue < 0.05:
                answer.append(True)
            else:
                answer.append(False)
        return answer

In [9]:
welch = WelchTTest()
welch_answers = welch.predict(experiments)
print(sklearn.metrics.confusion_matrix(create_target_series(experiments), welch_answers))

[[62935  2968]
 [20994 13103]]


In [10]:
sklearn.metrics.confusion_matrix([False, False, False, False, True, True, True, True],
                                [True, False, False, False, True, True, True, True])[0][1]

1

In [11]:
def type_1_error_ratio(y_true, y_pred):
    return sklearn.metrics.confusion_matrix(y_true, y_pred)[0][1] / len(y_true)

type_1_error_score = sklearn.metrics.make_scorer(type_1_error_ratio, greater_is_better=False)

## The target to beat...

For accuracy.

In [12]:
welch_accuracy = sklearn.metrics.accuracy_score(create_target_series(experiments), welch_answers)
welch_accuracy

0.76038

But you have to keep the type 1 error rate below this...

In [13]:
welch_type_1_error_ratio = type_1_error_ratio(create_target_series(experiments), welch_answers)
welch_type_1_error_ratio

0.02968

## Various stupid ways to beat it

Dummy gives us a baseline to be sure that the real models aren't just predicting the most common case.

In [14]:
dummy_scores = sklearn.model_selection.cross_validate(sklearn.dummy.DummyClassifier(strategy='most_frequent'), 
                                       create_feature_dataframe(experiments),
                                       create_target_series(experiments),
                                       scoring={'accuracy': 'accuracy', 
                                                'type_1_error_ratio': type_1_error_score}
                                      )
print("Mean accuracy:", dummy_scores['test_accuracy'].mean())
print("Mean type 1 error:", abs(dummy_scores['test_type_1_error_ratio'].mean()))

Mean accuracy: 0.6590299999999999
Mean type 1 error: 0.0


## Random forest 

It seems to beat Welch if there is enough training data.

In [15]:
%%time
rfc_scores = sklearn.model_selection.cross_validate(sklearn.ensemble.RandomForestClassifier(), 
                                       create_feature_dataframe(experiments),
                                       create_target_series(experiments),
                                       scoring={'accuracy': 'accuracy', 
                                                'type_1_error_ratio': type_1_error_score}
                                               )
print("Mean accuracy:", rfc_scores['test_accuracy'].mean())
print("Mean type 1 error:", abs(rfc_scores['test_type_1_error_ratio'].mean()))

Mean accuracy: 0.78466
Mean type 1 error: 0.030570000000000003
CPU times: user 3min 30s, sys: 664 ms, total: 3min 31s
Wall time: 3min 31s


In [16]:
if abs(rfc_scores['test_type_1_error_ratio'].mean()) < 0.05 and rfc_scores['test_accuracy'].mean() > welch_accuracy:
    print("Random forest classifier beat the Welch t-test while maintaining a low type 1 error ratio")

Random forest classifier beat the Welch t-test while maintaining a low type 1 error ratio


In [17]:
if abs(rfc_scores['test_type_1_error_ratio'].mean()) < welch_type_1_error_ratio:
    print("Random forest beat the Welch t-test on type 1 errors")

## Neighbour methods

In [18]:
%%time
knn_scores = sklearn.model_selection.cross_validate(sklearn.neighbors.KNeighborsClassifier(), 
                                       create_feature_dataframe(experiments),
                                       create_target_series(experiments),
                                       scoring={'accuracy': 'accuracy', 
                                                'type_1_error_ratio': type_1_error_score}
                                               )
print("Mean accuracy:", knn_scores['test_accuracy'].mean())
print("Mean type 1 error:", abs(knn_scores['test_type_1_error_ratio'].mean()))

Mean accuracy: 0.76083
Mean type 1 error: 0.08343
CPU times: user 49.1 s, sys: 127 ms, total: 49.2 s
Wall time: 49.3 s


In [19]:
if abs(knn_scores['test_type_1_error_ratio'].mean()) < 0.05 and knn_scores['test_accuracy'].mean() > welch_accuracy:
    print("Random forest classifier beat the Welch t-test while maintaining a low type 1 error ratio")

In [20]:
#%%time
#
#radnn_scores = sklearn.model_selection.cross_validate(sklearn.neighbors.RadiusNeighborsClassifier(), 
#                                       create_feature_dataframe(experiments),
#                                       create_target_series(experiments),
#                                       scoring={'accuracy': 'accuracy', 
#                                                'type_1_error_ratio': type_1_error_score}
#                                               )
#print("Mean accuracy:", radnn_scores['test_accuracy'].mean())
#print("Mean type 1 error:", abs(radnn_scores['test_type_1_error_ratio'].mean()))

In [21]:
#if abs(radnn_scores['test_type_1_error_ratio'].mean()) < 0.05 and radnn_scores['test_accuracy'].mean() > welch_accuracy:
#    print("Random forest classifier beat the Welch t-test while maintaining a low type 1 error ratio")

### Lost causes

You would think that SVM models would work really well; but they take too long to be practical. It's also
not clear whether they are actually improving with more data.

In [22]:
#%%time
#sklearn.model_selection.cross_validate(sklearn.svm.SVC(kernel='rbf', C=1e9), 
#                                       create_feature_dataframe(experiments),
#                                       create_target_series(experiments))['test_score'].mean()

In [23]:
#%%time
#sklearn.model_selection.cross_validate(sklearn.svm.SVC(kernel='poly', C=1e9, degree=2), 
##                                       create_feature_dataframe(experiments),
#                                       create_target_series(experiments))['test_score'].mean()

In [24]:
#sklearn.model_selection.cross_validate(sklearn.svm.LinearSVC(dual=True, C=1e9), 
#                                       create_feature_dataframe(experiments),
#                                       create_target_series(experiments))['test_score'].mean()